In [376]:
#габариты и координаты (в см)
from typing import Union

class Coords:
    def __init__(self, x: Union[int, float], y: Union[int, float], z: Union[int, float]):
        self.x = x
        self.y = y
        self.z = z
        
    def __str__(self):  
        return "(x:% s y:% s z:% s)" % (self.x, self.y, self.z)  #подумать над разделением

In [377]:
d = Coords("f", 2, 3)
print(d)

(x:f y:2 z:3)


In [378]:
# #размеры и расположение объектов
# from dataclasses import dataclass

# #координаты: середина дна гарнитура 
# #габариты: x - длина, y - высота, z - ширина 

# @dataclass
# class Kitchen_Object:
#     size: Coords
#     coords: Coords
    

In [379]:
#размеры и расположение объектов

#координаты: середина дна гарнитура 
#габариты: x - длина, y - высота, z - ширина 

class Kitchen_Object:
    def __init__(self, size: Coords, coords: Coords):
        self.size = size
        self.coords = coords
    
    #метод нахождения координат двух углов дна гарнитура (на одной из диагонали)
    @classmethod
    def corner_coords(self, size, coords):
        x = [coords.x - size.x/2, coords.z + size.z/2]
        z = [coords.x + size.x/2, coords.z - size.z/2]
        return x, z

#перенести сюда эти методы из класса Cabinet
#     def __if_bottom_overlap()
#     def __if_height_overlap()
#     def if_overlap()

In [380]:
from typing import Union

#пусть координаты - середина дна кухонного шкафа

class Cabinet(Kitchen_Object):
    def __init__(self, material, size, coords):
        super().__init__(size, coords)
        self.material = material
        
    def __repr__(self):  
        return "Cabinet(material: % s, size: % s, coords: % s)" % (self.material, self.size, self.coords) 

    @property
    def material(self):
        return self.__material
    
    @material.setter 
    def material(self, material: Union[str, int]):
        self.__material = material
        
# метод проверки на пересечение с другим шкаформ в параллелепипеде кухни
# NB: будем считать, что шкафы могут стоять только параллельно стенам кухни
    def __height_overlap(self, cabinet1: Cabinet, cabinet2: Cabinet):
        height1 = cabinet1.size.y
        height2 = cabinet2.size.y
        y1 = cabinet1.coords.y
        y2 = cabinet2.coords.y
        if y1 + height1 < y2:
            return False
        if y2 + height2 < y1:
            return False 
        return True
     
    def __bottom_overlap(self, cabinet1: Cabinet, cabinet2: Cabinet):
        coords1 = Kitchen_Object.corner_coords(cabinet1.size, cabinet1.coords) # координаты по обеим концам диагонали
        top1 = coords1[0] # координаты верхней точки (x;z)
        bottom1 = coords1[1] # координаты нижней точки (x;z)
        coords2 = Kitchen_Object.corner_coords(cabinet2.size, cabinet2.coords)
        top2 = coords2[0]
        bottom2 = coords2[1]
        if top1[0] == bottom1[0] or top1[1] == bottom1[1] or top2[0] == bottom2[0] or top2[1] == bottom2[1]: #площадь 0
            return False
        if top1[0] > bottom2[0] or top2[0] >  bottom1[0]: #
            return False
        if bottom1[1] > top2[0] or bottom2[1] > top1[1]:
            return False
        return True
    
    @classmethod
    def overlap(self, cabinet1: Cabinet, cabinet2: Cabinet):
        if self.__bottom_overlap(cabinet1, cabinet2) == False:
            return False
        if self.__height_overlap(cabinet1, cabinet2):
            return True

In [385]:
cab1 = Cabinet("wood", Coords(3,2,4), Coords(4,5,6))
cab1 = Cabinet("wood", Coords(3,2,4), Coords(4,5,6))
cab1.size.z
Cabinet.overlap(cab1, cab1)

TypeError: __bottom_overlap() missing 1 required positional argument: 'cabinet2'

In [260]:
class Appliance(Kitchen_Object):
    def __init__(self, name, size, coords):
        super().__init__(size, coords)
        self.__name = name

    @property
    def name(self):
        return self.__name
    
    @name.setter 
    def name(self, name: Union[str, int]):
        self.__name = name

In [369]:
class Kitchen:
    pass
    